In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

BRONZE_PATH = os.environ.get("BRONZE_PATH")
SILVER_PATH = os.environ.get("SILVER_PATH")
MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY")
MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY")

bronze_path = f"s3a:/{BRONZE_PATH}/vnstock3/stock_quote_history_daily/"
silver_path = f"s3a:/{SILVER_PATH}/vnstock3/daily_stock_prices/"

endpoint_url = "http://localhost:9000"

print(bronze_path)
print(silver_path)


# Setup Spark sesssion
spark = (SparkSession.builder
    .master("spark://spark-master:7077")
    .appName("DailyStockPriceProcessor")
    .getOrCreate()
)

spark

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType, IntegerType, DateType

In [ ]:
schema = StructType([
    StructField("time", StringType(), True),
    StructField("open", FloatType(), True),
    StructField("high", FloatType(), True),
    StructField("low", FloatType(), True),
    StructField("close", FloatType(), True),
    StructField("volume", IntegerType(), True),
    StructField("symbol", StringType(), True),
    StructField("loaded_timestamp", StringType(), True),
    StructField("updated_at", DateType(), True)
])

# Read the raw data from the bronze path in MinIO
raw_df = spark.read.schema(schema).json(bronze_path)